In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.trainer import Trainer
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.distribution.multivariate_gaussian import MultivariateGaussianOutput
from gluonts.model.predictor import Predictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from indycar.model.NaivePredictor import NaivePredictor
from indycar.model.ZeroPredictor import ZeroPredictor
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
import indycar.model.stint_simulator_shortterm_pitmodel as stint

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [18]:
def simulation(datasetid, testevent, taskid, runts, expid, predictionlen, 
               datamode, loopcnt, featuremode = stint.FEATURE_STATUS,
              pitmodel = 0):
    #
    # configurataion
    #
    # model path:  <_dataset_id>/<_task_id>-<trainid>/
    #_dataset_id = 'indy2013-2018-nocarid'
    stint.init(pitmodel)
    stint._dataset_id = datasetid
    stint._test_event = testevent
    #_test_event = 'Indy500-2019'

    stint._feature_mode = featuremode
    stint._context_ratio = 0.

    stint._task_id = taskid  # rank,laptime, the trained model's task
    stint._run_ts = runts   #COL_LAPTIME,COL_RANK
    stint._exp_id=expid  #rank, laptime, laptim2rank, timediff2rank... 

    stint._train_len = 40
    predictor = stint.load_model(predictionlen, 'oracle',trainid='2018')

    ret2 = {}
    for i in range(loopcnt):
        ret2[i] = stint.run_simulation_shortterm(predictor, predictionlen, stint.freq, datamode=datamode)

    acc = []
    for i in ret2.keys():
        df = ret2[i]
        _x = stint.get_evalret_shortterm(df)
        acc.append(_x)

    b = np.array(acc)
    print(np.mean(b, axis=0))
    
    #save keys
    stint._pitmodel.save_keys('pitmodel-keys.pickle')
    
    return b, ret2

In [28]:
def test_pitmodel(modelfile, testevent, runts, loopcnt=10):
    #pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle
    if runts == 'RANK':
        acc, ret = simulation('indy2013-2018-nocarid-context40', testevent, 
                    'rank',stint.COL_RANK,'rank',
                   2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                      pitmodel=modelfile)        
    else:
        acc, ret = simulation('indy2013-2018-nocarid-context40', testevent, 
                        'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                       2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                          pitmodel=modelfile)

    print(np.mean(acc, axis=0)  )

### test low mode

In [16]:
acc, ret = simulation('indy2013-2018', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,1, pitmodel=1)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init pitmodel as PitModelSimple
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.7287234042553191, mae=1.5602996254681647, rmse=10.925468164794008,r2=0.847462547292267, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
[ 0.7287234   1.56029963 10.92546816  0.84746255]
save keys to pitmodel-keys.pickle.


In [19]:
acc, ret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,10, pitmodel=1)

np.mean(acc, axis=0)    

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init pitmodel as PitModelSimple
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.7287234042553191, mae=1.3808988764044945, rmse=10.080149812734083,r2=0.8592645777595582, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.75, mae=1.3632958801498127, rmse=10.107490636704119,r2=0.8588828550195882, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7340425531914894, mae=1.4359550561797754, rmse=11.026591760299626,r2=0.8460506960622409, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7819148936170213, mae=1.4067415730337078, rmse=10.686142322097378,r2=0.8508039285366618, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7287234042553191, mae=1.3966292134831462, rmse=10.644569288389514,r2=0.8513843562645613, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7553191489361702, mae=1.3677902621722846, rmse=10.0153558052434

array([ 0.74308511,  1.39651685, 10.40808989,  0.854686  ])

In [22]:
acc, ret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'rank',stint.COL_RANK,'rank',
                   2, stint.MODE_ORACLE_LAPONLY,10, 
                      pitmodel=1)

np.mean(acc, axis=0) 

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init pitmodel as PitModelSimple
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.7606382978723404, mae=1.3367041198501872, rmse=10.076404494382022,r2=0.8593168685458554, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7553191489361702, mae=1.3531835205992508, rmse=10.110861423220975,r2=0.8588357933119206, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7446808510638298, mae=1.357677902621723, rmse=10.21123595505618,r2=0.8574344002391542, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.723404255319149, mae=1.3565543071161048, rmse=10.198876404494381,r2=0.8576069598339351, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7553191489361702, mae=1.347565543071161, rmse=10.105992509363295,r2=0.8589037713341071, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7340425531914894, mae=1.3269662921348315, rmse=9.7348

array([ 0.74680851,  1.34838951, 10.14183521,  0.85840335])

In [7]:
acc, ret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,10)

np.mean(acc, axis=0)    

INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init pitmodel as PitModelSimple
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.7553191489361702, mae=1.3614232209737829, rmse=9.75692883895131,r2=0.8637772726170116, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7553191489361702, mae=1.3464419475655431, rmse=9.806367041198502,r2=0.8630870342378878, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7287234042553191, mae=1.3842696629213482, rmse=10.260299625468164,r2=0.8567493909386601, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7287234042553191, mae=1.3737827715355806, rmse=9.913483146067415,r2=0.8615915177497862, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7606382978723404, mae=1.3749063670411985, rmse=9.912734082397003,r2=0.8616019759070457, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7446808510638298, mae=1.3636704119850187, rmse=9.71048

array([ 0.74680851,  1.37153558, 10.01355805,  0.86019431])

In [20]:
acc, ret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,10, 
                      pitmodel='pitmodel-m60-mlp-e2000-l10-10-5-student-d0.1.pickle')

np.mean(acc, axis=0)    

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m60-mlp-e2000-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.7340425531914894, mae=1.4737827715355805, rmse=11.305992509363296,r2=0.8421498034044654, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.75, mae=1.506741573033708, rmse=12.033333333333333,r2=0.8319949327055378, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7340425531914894, mae=1.5149812734082397, rmse=11.677528089887641,r2=0.8369625574037773, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7021276595744681, mae=1.4865168539325844, rmse=11.740449438202248,r2=0.8360840721939835, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7340425531914894, mae=1.497003745318352, rmse=11.920224719101123,r2=0.833574114451715, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: ac

array([ 0.73085106,  1.48730337, 11.66250936,  0.83717224])

In [21]:
acc, ret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'rank',stint.COL_RANK,'rank',
                   2, stint.MODE_ORACLE_LAPONLY,10, 
                      pitmodel='pitmodel-m60-mlp-e2000-l10-10-5-student-d0.1.pickle')

np.mean(acc, axis=0)   

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m60-mlp-e2000-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.723404255319149, mae=1.3584269662921349, rmse=9.975280898876404,r2=0.8607287197758815, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7659574468085106, mae=1.353558052434457, rmse=10.010486891385767,r2=0.8602371863846872, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7393617021276596, mae=1.3707865168539326, rmse=10.177902621722847,r2=0.8578997882371998, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7553191489361702, mae=1.3610486891385767, rmse=10.017977528089888,r2=0.8601326048120927, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7287234042553191, mae=1.3910112359550562, rmse=10.482022471910112,r2=0.8536537763898624, acc_naive=0.7180851063829787, mae_naive=1.33932584269

array([ 0.74202128,  1.36550562, 10.15048689,  0.85828256])

In [26]:
acc, ret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2019', 
                    'rank',stint.COL_RANK,'rank',
                   2, stint.MODE_ORACLE_LAPONLY,10, 
                      pitmodel=1)

np.mean(acc, axis=0)  

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init pitmodel as PitModelSimple
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
pred: acc=0.7606382978723404, mae=1.1970154556759638, rmse=9.0506306626399,r2=0.881800336506643, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.7659574468085106, mae=1.2023449991117428, rmse=9.161129863208386,r2=0.8803572361515049, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.7446808510638298, mae=1.20305560490318, rmse=9.107479125954876,r2=0.8810579054557681, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.7287234042553191, mae=1.2005684846331497, rmse=9.183513945638657,r2=0.8800649039252229, acc_naive=0.7340425

array([0.74042553, 1.19833008, 9.09728193, 0.88119108])

In [27]:
acc, ret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2019', 
                    'rank',stint.COL_RANK,'rank',
                   2, stint.MODE_ORACLE_LAPONLY,10, 
                      pitmodel=0)

np.mean(acc, axis=0)  

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init pitmodel as PitModelSimple
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
pred: acc=0.7553191489361702, mae=1.1948836383016521, rmse=9.027535974418191,r2=0.882101949121061, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.7446808510638298, mae=1.1966601527802452, rmse=9.067329898738675,r2=0.8815822473854485, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.723404255319149, mae=1.20305560490318, rmse=9.092201101438977,r2=0.8812574338006907, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.7393617021276596, mae=1.1824480369515011, rmse=8.89856102327234,r2=0.8837863395677334, acc_naive=0.7340425

array([0.74361702, 1.18852372, 8.97949902, 0.8827293 ])

In [23]:
acc, ret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2019', 
                    'rank',stint.COL_RANK,'rank',
                   2, stint.MODE_ORACLE_LAPONLY,10, 
                      pitmodel='pitmodel-m60-mlp-e2000-l10-10-5-student-d0.1.pickle')

np.mean(acc, axis=0)   

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m60-mlp-e2000-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
pred: acc=0.7180851063829787, mae=1.2027003020074614, rmse=9.012968555693728,r2=0.8822921970778477, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.7393617021276596, mae=1.193107123823059, rmse=8.978504174809025,r2=0.8827422959024406, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.7127659574468085, mae=1.2240184757505774, rmse=9.484455498312311,r2=0.8761346595496537, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.7393617021276596, mae=1.209451057

array([0.74042553, 1.20479659, 9.12620359, 0.88081337])

In [24]:
acc, ret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2019', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,10, 
                      pitmodel='pitmodel-m60-mlp-e2000-l10-10-5-student-d0.1.pickle')

np.mean(acc, axis=0)    

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m60-mlp-e2000-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
pred: acc=0.6914893617021277, mae=1.368271451412329, rmse=10.354592289927163,r2=0.8647708242771993, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.675531914893617, mae=1.307869959140167, rmse=9.465979747734943,r2=0.876375949641188, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.6808510638297872, mae=1.2947237519985788, rmse=9.857168235921122,r2=0.871267095972355, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.6648936170212766, mae=1.366139634038

array([0.6856383 , 1.32154912, 9.84888968, 0.87137521])

### test

In [36]:
np.mean(acc, axis=0)

array([ 0.70904255,  1.56734082, 10.99475655,  0.84649517])

In [12]:
139./188

0.7393617021276596

In [27]:
stint.get_evalret_shortterm(df)

pred: acc=0.6968085106382979, mae=1.5872659176029962,45.08510638297872, rmse=11.434831460674157,r2=0.8403510003558398, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292, 38.04255319148936


(0.6968085106382979,
 1.5872659176029962,
 11.434831460674157,
 0.8403510003558398)

In [15]:
df

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,1,10,3.0,3.0,0.0,0,3.0,0.0,0
1,3,10,7.0,7.0,0.0,0,7.0,0.0,0
2,4,10,12.0,12.0,0.0,0,12.0,0.0,0
3,6,10,15.0,15.0,0.0,0,15.0,0.0,0
4,7,10,28.0,28.0,0.0,0,29.0,1.0,1
...,...,...,...,...,...,...,...,...,...
5335,60,197,15.0,15.0,0.0,0,15.0,0.0,0
5336,64,197,16.0,16.0,0.0,0,14.0,-2.0,-1
5337,66,197,10.0,10.0,0.0,0,16.0,6.0,1
5338,88,197,13.0,13.0,0.0,0,12.0,-1.0,-1


In [23]:
np.sum(df[['diff']].values.reshape(-1))

[autoreload of indycar.model.stint_simulator_shortterm failed: Traceback (most recent call last):
  File "/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<f

0.0

In [37]:
acc, ret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,1)
df  = ret[0]
stint.get_evalret_shortterm(df)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.7287234042553191, mae=1.3966292134831462, rmse=10.328838951310862,r2=0.8557924695494202, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
[ 0.7287234   1.39662921 10.32883895  0.85579247]
pred: acc=0.7287234042553191, mae=1.3966292134831462, rmse=10.328838951310862,r2=0.8557924695494202, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292


(0.7287234042553191,
 1.3966292134831462,
 10.328838951310862,
 0.8557924695494202)

In [29]:
#def test_pitmodel(modelfile, testevent, runts, loopcnt=10):
test_pitmodel('pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'RANK')

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.7021276595744681, mae=1.4232209737827715, rmse=10.70374531835206,r2=0.8505581618410647, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7074468085106383, mae=1.4389513108614231, rmse=10.834831460674158,r2=0.8487279843206605, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7021276595744681, mae=1.4344569288389513, rmse=10.96816479400749,r2=0.8468664323284782, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7127659574468085, mae=1.4265917602996254, rmse=10.789887640449438,r2=0.8493554737562277, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.6968085106382979, mae=1.4340823970037453, rmse=10.894756554307117,r2=0.8478913317399044, acc_naive=0.7180851063829787, mae_naive=1.33932584269

In [31]:
test_pitmodel('pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'RANK')

INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
pred: acc=0.7127659574468085, mae=1.268786640611121, rmse=9.469888079587848,r2=0.8763249075064403, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.75, mae=1.270563155089714, rmse=9.501154734411086,r2=0.8759165704284592, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.6648936170212766, mae=1.2844199680227393, rmse=9.609877420500977,r2=0.8744966710436608, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.7393617021276596, mae=1.2719843666725883, rmse=9.

In [36]:
test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'RANK')

INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
pred: acc=0.7659574468085106, mae=1.3292134831460674, rmse=9.690262172284644,r2=0.8647080486131028, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7606382978723404, mae=1.3370786516853932, rmse=9.88314606741573,r2=0.862015073118794, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.75, mae=1.341573033707865, rmse=10.004494382022472,r2=0.8603208516427628, acc_naive=0.7180851063829

In [35]:
%debug

> /N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py(1799)update_onets()
   1797         pred_pit_laps = _pitmodel.predict(caution_laps_instint, laps_instint)
   1798 
-> 1799         nextpos = curpos + pred_pit_laps - laps_instint
   1800 
   1801         if nextpos >= totallen:

ipdb> p _pitmodel.keys()
*** TypeError: 'dict' object is not callable
ipdb> p _pitmodel.keys
{}
ipdb> p _pitmodel.model.keys()
dict_keys(['0-0', '0-1', '0-2', '0-3', '0-4', '0-5', '0-6', '0-7', '0-8', '0-9', '0-10', '0-11', '0-12', '0-13', '0-14', '0-15', '0-16', '0-17', '0-18', '0-19', '0-20', '0-21', '0-22', '0-23', '0-24', '0-25', '0-26', '0-27', '0-28', '0-29', '0-30', '0-31', '0-32', '0-33', '0-34', '0-35', '0-36', '0-37', '0-38', '0-39', '0-40', '0-41', '0-42', '0-43', '0-44', '0-45', '0-46', '0-47', '0-48', '0-49', '0-50', '0-51', '0-52', '0-53', '0-54', '0-55', '0-56', '0-57', '0-58', '0-59', '0-60', '0-61', '0-62', '0-63', '0-64', '1-1', '1-2', '1-3', '1-4', '1-

ipdb> p _pitmodel.model['14-45']
array([], shape=(2, 0), dtype=float64)
ipdb> quit


In [37]:
test_pitmodel('pitmodel-m60-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'RANK')

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m60-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
pred: acc=0.7393617021276596, mae=1.1842245514300942, rmse=8.790904245869603,r2=0.8851923183703279, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.7287234042553191, mae=1.1998578788417125, rmse=9.142654112631018,r2=0.8805985262430391, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.75, mae=1.1863563688044056, rmse=8.849529223663174,r2=0.884426686349113, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.7553191489361702, mae=1.1970154556759638, r

In [38]:
test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'TIMEDIFF')
test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'TIMEDIFF')

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
pred: acc=0.776595744680851, mae=1.3644194756554306, rmse=9.961048689138577,r2=0.860927424763811, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7446808510638298, mae=1.3318352059925094, rmse=9.822471910112359,r2=0.8628621838568096, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7446808510638298, mae=1.352434456928839, rmse=10.367041198501873,r2=0.8552591035291882, acc_naive=0

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
pred: acc=0.7021276595744681, mae=1.2360987742050098, rmse=8.706342156688576,r2=0.8862966845585043, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.6914893617021277, mae=1.2151359033576123, rmse=8.676496713448214,r2=0.8866864608602136, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.7021276595744681, mae=1.2158465091490496, rmse=8.516610410374845,r2=0.8887745481907994, acc_naive=0.7340425531914894, mae_naive=1.1625510747912595
pred: acc=0.6861702127659575, mae=1.2460